# 既約多項式をチェックする
#### 先ずは環境のチェックと **Nemo** などの**Pkg**利用設定

In [1]:
println(versioninfo())
using Nemo,Plots,LaTeXStrings,Latexify,Combinatorics,Dates,Primes

Julia Version 1.10.4
Commit 48d4fd48430 (2024-06-04 10:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M3
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
nothing

Welcome to Nemo version building

Nemo comes with absolutely no warranty whatsoever


#### 有限体$GF(p)$の3次拡大から有限射影平面を拵えて魔円陣を構成する。

In [2]:
p=7
q=p^2+p+1
Fp=GF(p)
println(p,",",q,", size = ",length(Fp))
println(join(Fp,", "))

7,57, size = 7
0, 1, 2, 3, 4, 5, 6


#### $Fp$上の多項式環の剰余環によって3次拡大を実現する。  
そのために先ず, $Fp$係数の3次の多項式を総浚えして, 既約多項式の候補を挙げる。  

In [3]:
Fpx, x = Fp["x"]
IPlst=[]
for a ∈ Fp, b ∈ Fp, c ∈ Fp
    f(x)=x^3+a*x^2+b*x+c
    chk=true
    for e ∈ Fp
        if f(e)==0
            chk=false
            break
        end
    end
    if chk
        push!(IPlst,f(x))
    end
end
if length(IPlst)>20
    println(join(IPlst[1:10],", "),"...",join(IPlst[end-9:end],", "))
else
    println(join(IPlst,", "))
end
println("既約多項式の候補の数 = ",length(IPlst))

x^3 + 2, x^3 + 3, x^3 + 4, x^3 + 5, x^3 + x + 1, x^3 + x + 6, x^3 + 2*x + 1, x^3 + 2*x + 6, x^3 + 3*x + 2, x^3 + 3*x + 5...x^3 + 6*x^2 + 3*x + 2, x^3 + 6*x^2 + 3*x + 6, x^3 + 6*x^2 + 4*x + 1, x^3 + 6*x^2 + 4*x + 4, x^3 + 6*x^2 + 5*x + 3, x^3 + 6*x^2 + 5*x + 4, x^3 + 6*x^2 + 5*x + 5, x^3 + 6*x^2 + 5*x + 6, x^3 + 6*x^2 + 6*x + 2, x^3 + 6*x^2 + 6*x + 4
既約多項式の候補の数 = 112


#### 生成元を確認して, 魔円陣を構成する
確認の仕方は, 既に求めてある既約多項式で, 有限体を構成し, その生成元を$\alpha$としておいて, 
$\alpha^i$を列挙していき, $\alpha^i=1$となったら抜けてチェックする。  
このとき, $\alpha^i$のリストの長さが, $p^3-1$になっていれば **生成元** と確認できたとする。  
生成元の候補としては, $\alpha$だけで良いのだろうか...。  
$\alpha$だけでなく, $\alpha+1,\ \alpha+2,\ \cdots$などもチェックするべきなのか...
$\alpha+1,\ \cdots$などを候補に入れると何故か上手くいかなかったので, ここでは取りやめた...。

#### 生成元の確認をもう少し効率よくするには...
$\alpha^i$を全部調べなくても, $1$ になる可能性があるのは, $i$ が $p^3-1$ の約数の場合だけなので, 順に調べずに先ず $p^3-1$ の約数の配列を拵える。

In [4]:
pp=p^3-1
sqrpp=Int(floor(sqrt(pp)))
println(pp,",",sqrpp)
pf=[]
for i=1:sqrpp
    if rem(pp,i)==0
        push!(pf,i)
        push!(pf,Int(pp/i))
    end
end
sort!(unique!(pf))
pop!(pf)
popfirst!(pf)
println(join(pf,","))

342,18
2,3,6,9,18,19,38,57,114,171


#### 更に約数全部チェックしなくても良いやんとなったけど... 駄目みたい... 

In [5]:
#pp=p^3-1
#sqrpp=Int(floor(sqrt(pp)))
#println(pp,",",sqrpp)
#pf=[]
#for i=1:sqrpp
#    if rem(pp,i)==0 && isprime(i)
#        push!(pf,i)
#        push!(pf,Int(pp/i))
#    end
#end
#sort!(unique!(pf))
#pop!(pf)
#popfirst!(pf)
#println(join(pf,","))

そうして得られた約数のリストに含まれる $i$ に対して, $\alpha^i$が $1$ になるかどうかを調べれば...

In [6]:
Fpt=[]
Fptl=[]
α=0
chk=false
r=0
for k=1:length(IPlst)
    Fpt,α=finite_field(IPlst[k],"α")
    for r=0:0#p-1
#        Fptl=[]
#        push!(Fptl,1)
#        for i=1:length(Fpt)
        chkA=true
        for i ∈ pf
            e=(α+r)^i
            if e==1
                chkA=false
                break
#            else
#                push!(Fptl,e)
            end
        end
        if chkA
#        println(k," r=",r," : size = ",length(Fptl), ", p^3-1 = ",p^3-1,", ",IPlst[k])
#        if length(unique(Fptl))==p^3-1
            println(Fpt)
            println(k,", r=",r," : ",IPlst[k])
            chk=true
            break
        end
    end
    if chk
        break
    end
end
#println(join(Fptl[1:10],", "),"...",join(Fptl[end-9:end],", "))
pdsl=[0]
for i=1:length(Fpt)
    if coeff((α+r)^i,2)==0
#        println(i,": ",α^i)
        push!(pdsl,i)
    end
end
println(pdsl[1:10],"...",pdsl[end-9:end])
bbl=[]
for i=1:p+1
    push!(bbl,pdsl[i+1]-pdsl[i])
end
println("(",join(bbl,","),")")

Finite field of degree 3 and characteristic 7
9, r=0 : x^3 + 3*x + 2
[0, 1, 3, 13, 32, 36, 43, 52, 57, 58]...[285, 286, 288, 298, 317, 321, 328, 337, 342, 343]
(1,2,10,19,4,7,9,5)


#### できた魔円陣を元に, （あれば）他の同サイズの魔円陣を列挙する。
(巡回)完全差集合に変換して, 整数倍して, ソートかけて, 見つけていく。

In [7]:
cds=[0]
for i=1:length(bbl)
    push!(cds,cds[end]+bbl[i])
end
println("Perfect Difference Set = (",join(cds,","),")")
σ=p^2+p+1
println("σ = ",σ)
otherlst=[]
for i=1:σ-1
#    println(sort!((((cds.*i).-1).%σ).+1))
    cdsa=sort!((((cds.*i).-1).%σ).+1)
    push!(otherlst,cdsa)
end
oblst=[]
for lst ∈ otherlst
    obll=[]
    for i=2:length(lst)
        push!(obll,lst[i]-lst[i-1])
    end
#    sort!(obll)
    while obll[1]>1
        circshift!(obll,-1)
    end
    if obll[2]>obll[end]
        obll=[obll[1];reverse(obll[2:end])]
    end
#    println("(",join(lst,","),") --> (",join(obll,","),")")
    if !(obll ∈ oblst) && !(0 ∈ obll)
        push!(oblst,obll)
    end
end
println("同じサイズの魔円陣の数は, ",length(oblst))
sort!(oblst, lt= (a,b)->a[2]*10+a[3]<b[2]*10+b[3])
for bll ∈ oblst
    println("(",join(bll,","),")")
end

Perfect Difference Set = (0,1,3,13,32,36,43,52,57)
σ = 57
同じサイズの魔円陣の数は, 6
(1,2,10,19,4,7,9,5)
(1,3,5,11,2,12,17,6)
(1,3,8,2,16,7,15,5)
(1,4,2,10,18,3,11,8)
(1,4,22,7,3,6,2,12)
(1,6,12,4,21,3,2,8)


#### 次は$GF(p^n)$の3次拡大で有限射影平面を拵えて, 魔円陣を構成する。
基本的に同じことをするのだけど... 先ず$GF(p^n)$を構成する時点で, 生成元 $\alpha$ が登場する。

In [8]:
n=3
q=p^n
println("p = ",p,", n = ",n,", q=p^n= ",q)
Fp,α=finite_field(p,n,"α")
for e ∈ Fp
    print(e,", ")
end
print("\n")
println("Fpのサイズは, ",length(Fp))

p = 7, n = 3, q=p^n= 343
0, 1, 2, 3, 4, 5, 6, α, α + 1, α + 2, α + 3, α + 4, α + 5, α + 6, 2*α, 2*α + 1, 2*α + 2, 2*α + 3, 2*α + 4, 2*α + 5, 2*α + 6, 3*α, 3*α + 1, 3*α + 2, 3*α + 3, 3*α + 4, 3*α + 5, 3*α + 6, 4*α, 4*α + 1, 4*α + 2, 4*α + 3, 4*α + 4, 4*α + 5, 4*α + 6, 5*α, 5*α + 1, 5*α + 2, 5*α + 3, 5*α + 4, 5*α + 5, 5*α + 6, 6*α, 6*α + 1, 6*α + 2, 6*α + 3, 6*α + 4, 6*α + 5, 6*α + 6, α^2, α^2 + 1, α^2 + 2, α^2 + 3, α^2 + 4, α^2 + 5, α^2 + 6, α^2 + α, α^2 + α + 1, α^2 + α + 2, α^2 + α + 3, α^2 + α + 4, α^2 + α + 5, α^2 + α + 6, α^2 + 2*α, α^2 + 2*α + 1, α^2 + 2*α + 2, α^2 + 2*α + 3, α^2 + 2*α + 4, α^2 + 2*α + 5, α^2 + 2*α + 6, α^2 + 3*α, α^2 + 3*α + 1, α^2 + 3*α + 2, α^2 + 3*α + 3, α^2 + 3*α + 4, α^2 + 3*α + 5, α^2 + 3*α + 6, α^2 + 4*α, α^2 + 4*α + 1, α^2 + 4*α + 2, α^2 + 4*α + 3, α^2 + 4*α + 4, α^2 + 4*α + 5, α^2 + 4*α + 6, α^2 + 5*α, α^2 + 5*α + 1, α^2 + 5*α + 2, α^2 + 5*α + 3, α^2 + 5*α + 4, α^2 + 5*α + 5, α^2 + 5*α + 6, α^2 + 6*α, α^2 + 6*α + 1, α^2 + 6*α + 2, α^2 + 6*α + 3, α^2 + 6*

#### 同様に既約多項式の候補を列挙するのだが, この時点で係数に $\alpha$ が出てくる。

In [9]:
start = Dates.now()
Fpx, x = Fp["x"]
IPlst=[]
for a ∈ Fp, b ∈ Fp, c ∈ Fp
    f(x)=x^3+a*x^2+b*x+c
    chk=true
    for e ∈ Fp
        if f(e)==0
            chk=false
            break
        end
    end
    if chk && (a!=0 || b!=0) # ちょっと注文付けてみた... 
        push!(IPlst,f(x))
    end
#    if length(IPlst)>2000
    if Dates.now() - start > Millisecond(2000)
        break
    end
end
println(length(IPlst))
println(join(IPlst[1:10],",")," ... ",join(IPlst[end-9:end],","))
dtime = Dates.now() - start
println("かかった時間: $dtime")

2337
x^3 + x + α + 3,x^3 + x + α + 6,x^3 + x + 2*α,x^3 + x + 2*α + 3,x^3 + x + 3*α + 2,x^3 + x + 3*α + 3,x^3 + x + 3*α + 4,x^3 + x + 3*α + 5,x^3 + x + 3*α + 6,x^3 + x + 4*α + 1 ... x^3 + 3*α*x + 3*α^2 + 1,x^3 + 3*α*x + 3*α^2 + 2,x^3 + 3*α*x + 3*α^2 + 3,x^3 + 3*α*x + 3*α^2 + α + 1,x^3 + 3*α*x + 3*α^2 + α + 3,x^3 + 3*α*x + 3*α^2 + 2*α,x^3 + 3*α*x + 3*α^2 + 2*α + 2,x^3 + 3*α*x + 3*α^2 + 2*α + 5,x^3 + 3*α*x + 3*α^2 + 2*α + 6,x^3 + 3*α*x + 3*α^2 + 3*α + 3
かかった時間: 2005 milliseconds


#### 既約多項式で3次拡大しながら, 生成元を確認して, 魔円陣を構成する
生成元としては, $\omega,\ \omega+1,\ \omega+2,\ \cdots,\ \omega+(p-1)$あたりが想定されるのだろうなぁ。

#### 生成元の確認を効率よくする為に, 先ず $q^3-1$ の約数の配列を拵える。$q=p^n$である。

In [10]:
q=p^n
qq=q^3-1
sqrqq=Int(floor(sqrt(qq)))
println(p,", ",n,", ",q,", ",qq,", ",sqrqq)
qf=[]
for i=1:sqrqq
    if rem(qq,i)==0
        push!(qf,i)
        push!(qf,Int(qq/i))
    end
end
sort!(unique!(qf))
pop!(qf)
popfirst!(qf)
println(qq," の約数は, ",join(qf,","))

7, 3, 343, 40353606, 6352
40353606 の約数は, 2,3,6,9,18,19,27,37,38,54,57,74,111,114,171,222,333,342,513,666,703,999,1026,1063,1406,1998,2109,2126,3189,4218,6327,6378,9567,12654,18981,19134,20197,28701,37962,39331,40394,57402,60591,78662,117993,121182,181773,235986,353979,363546,545319,707958,747289,1061937,1090638,1494578,2123874,2241867,4483734,6725601,13451202,20176803


In [11]:
start = Dates.now()
ω=0
r=0
chk=false
ωω=ω
for k=1:length(IPlst)
    Fpt,ω=finite_field(IPlst[k],"ω")
    for r=0:p-1
        ωω=ω+r
        Fptl=[]
        push!(Fptl,1)
#        for i=1:length(Fpt)
        chkA=true
        i=0
        for i ∈ qf
#    e=ω^i
            e=ωω^i
#    println(i,": ",e)
            if e==1
                chkA=false
                break
#            else
#                push!(Fptl,e)
            end
        end
        dtime = Dates.now() - start
        println(k,", ",ωω," : i = ",i, ", p^3-1 = ",(p^n)^3-1,", かかった時間: $dtime millis")
#        if length(Fptl)==(p^n)^3-1
        if chkA
#            println(Fpt)
            println(k,", r=",r," : ",IPlst[k])
            chk=true
            break
        end
    end
    if chk
        break
    end
end
#println(join(Fptl[1:10],", ")," ... ",join(Fptl[end-9:end],", "))
dtime = Dates.now() - start
println("かかった時間: $dtime")
pdsl=[0]
for i=1:qq
    if coeff(ωω^i,2)==0
#    if coeff(ω^i,2)==0
#        println(i,": ",α^i)
        push!(pdsl,i)
    end
end
println(join(pdsl[1:10],","),"...",join(pdsl[end-9:end],","))
bbl=[]
for i=1:p^n+1
    push!(bbl,pdsl[i+1]-pdsl[i])
end
println(join(bbl,","))
println("size = ",length(bbl))
dtime = Dates.now() - start
println("かかった時間: $dtime")

1, ω : i = 0, p^3-1 = 40353606, かかった時間: 30 milliseconds millis
1, r=0 : x^3 + x + α + 3
かかった時間: 182 milliseconds
0,1,3,80,536,2259,2614,2661,2822,2847...40350707,40351139,40351464,40351714,40351784,40352042,40352324,40352615,40353556,40353606
1,2,77,456,1723,355,47,161,25,713,31,58,91,141,237,257,190,156,87,1027,326,305,197,660,138,124,81,409,277,1529,218,196,518,681,361,20,1257,99,55,453,142,11,362,181,120,1022,617,219,750,564,619,797,1336,350,7,14,297,242,9,482,624,377,24,72,608,245,125,419,207,52,264,973,884,171,103,308,529,319,664,111,700,1599,13,54,1448,43,146,1144,35,622,441,405,5,724,41,263,195,753,1498,39,17,65,247,40,73,331,117,16,194,322,34,175,157,344,78,1167,198,109,1171,1767,217,223,86,97,637,584,380,23,249,228,166,49,98,697,172,1210,150,439,44,216,845,63,162,703,389,609,1040,471,654,421,496,832,60,273,90,29,255,200,943,115,472,199,68,212,347,12,148,4,298,879,32,136,38,191,275,281,36,151,306,513,112,1017,234,22,85,1038,461,37,303,315,100,238,213,399,110,173,214,8,108,19,12

In [12]:
start = Dates.now()
bblw=[bbl;bbl]
#println(bblw)
suml=[]
for b=1:length(bbl)-1
    for i=1:length(bbl)
        sum=0
        for j=i:i+b-1
            sum=sum+bblw[j]
        end
        push!(suml,sum)
    end
end
push!(suml,sum(bbl))
println(join(sort(unique(suml))[1:20],",")," ... ",join(sort(unique(suml))[end-10:end],",")," : ",length(suml))
dtime = Dates.now() - start
println("かかった時間: $dtime")

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20 ... 117983,117984,117985,117986,117987,117988,117989,117990,117991,117992,117993 : 117993
かかった時間: 1053 milliseconds


In [13]:
cds=[0]
for i=1:length(bbl)
    push!(cds,cds[end]+bbl[i])
end
println("Perfect Difference Set = (",join(cds,","),")")
σ=(p^n)^2+(p^n)+1
println("σ = ",σ)
otherlst=[]
for i=1:σ-1
#    println(sort!((((cds.*i).-1).%σ).+1))
    cdsa=sort!((((cds.*i).-1).%σ).+1)
    push!(otherlst,cdsa)
end
oblst=[]
for lst ∈ otherlst
    obll=[]
    for i=2:length(lst)
        push!(obll,lst[i]-lst[i-1])
    end
#    sort!(obll)
    while obll[1]>1
        circshift!(obll,-1)
    end
    if obll[2]>obll[end]
        obll=[obll[1];reverse(obll[2:end])]
    end
#    println("(",join(lst,","),") --> (",join(obll,","),")")
    if !(obll ∈ oblst) && !(0 ∈ obll)
        push!(oblst,obll)
    end
end
println("同じサイズ ",length(oblst[1])," な魔円陣の数は, ",length(oblst))
sort!(oblst, lt= (a,b)->a[2]*10+a[3]<b[2]*10+b[3])
if length(oblst)>20
    for bll ∈ oblst[1:10]
        println("(",join(bll,","),")")
    end
    println("...")
    for bll ∈ oblst[end-9:end]
        println("(",join(bll,","),")")
    end
end


Perfect Difference Set = (0,1,3,80,536,2259,2614,2661,2822,2847,3560,3591,3649,3740,3881,4118,4375,4565,4721,4808,5835,6161,6466,6663,7323,7461,7585,7666,8075,8352,9881,10099,10295,10813,11494,11855,11875,13132,13231,13286,13739,13881,13892,14254,14435,14555,15577,16194,16413,17163,17727,18346,19143,20479,20829,20836,20850,21147,21389,21398,21880,22504,22881,22905,22977,23585,23830,23955,24374,24581,24633,24897,25870,26754,26925,27028,27336,27865,28184,28848,28959,29659,31258,31271,31325,32773,32816,32962,34106,34141,34763,35204,35609,35614,36338,36379,36642,36837,37590,39088,39127,39144,39209,39456,39496,39569,39900,40017,40033,40227,40549,40583,40758,40915,41259,41337,42504,42702,42811,43982,45749,45966,46189,46275,46372,47009,47593,47973,47996,48245,48473,48639,48688,48786,49483,49655,50865,51015,51454,51498,51714,52559,52622,52784,53487,53876,54485,55525,55996,56650,57071,57567,58399,58459,58732,58822,58851,59106,59306,60249,60364,60836,61035,61103,61315,61662,61674,61822,61826,621